In [3]:
import os
os.chdir('..')

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import gm
import dill
import gzip
from pathlib import Path
import importlib
from src.data import get_gwi, get_cmip6, sample_glaciers_from_gcms, get_rgi, get_cmip6_run_attrs, get_cmip6_lm, sample_glaciers, get_lmr
import src.data as data
from src.flowline import get_flowline_geom
from src.climate import detrend_gcm, extend_ts, prepend_ts
import xarray as xr
import multiprocess as mp
from itertools import product
from src.util import dict_key_from_value
import xrft

In [8]:
import config

importlib.reload(config)
from config import cfg, ROOT


def temp_stable(yr):
    return 0


rgiid = list({k: v for k, v in cfg['glaciers'].items() if v['name'] == 'Argentiere'}.keys())[0]
params = cfg['glaciers'][rgiid]['flowline_params']

print(params)

{'ss.T0': 20.9, 'ss.noise.T0': 20.86, 'P0': 2, 'sigP': 0.55, 'sigT': 0.8, 'sigb': 0.71, 'L0': {'value': 10283, 'year': 1847, 'source': 'Leclerq'}, 'ss.noise.s_ba.profile': 'features/flowline/flowline2d_RGI60-11.03638.ss.noise.s_ba.pickle'}


In [9]:
lmr = get_lmr()

In [10]:
lmr

<xarray.Dataset>
Dimensions:    (time: 2001, latitude: 91, longitude: 180, MCrun: 20)
Coordinates:
  * time       (time) int64 0 1 2 3 4 5 6 ... 1994 1995 1996 1997 1998 1999 2000
  * latitude   (latitude) float32 -90.0 -88.0 -86.0 -84.0 ... 86.0 88.0 90.0
  * longitude  (longitude) float32 0.0 2.0 4.0 6.0 ... 352.0 354.0 356.0 358.0
Dimensions without coordinates: MCrun
Data variables:
    prate      (time, MCrun, latitude, longitude) float64 0.0001039 ... 0.008377
    air        (time, MCrun, latitude, longitude) float32 ...
Attributes:
    description:  Last Millennium Reanalysis climate field reconstruction for...
    experiment:   productionFinal2_gisgpcc_ccms4_LMRdbv1.1.0_pr
    comment:      File contains ensemble-mean values for each Monte-Carlo rec...

In [11]:
T = lmr.air
T

<xarray.DataArray 'air' (time: 2001, MCrun: 20, latitude: 91, longitude: 180)>
[655527600 values with dtype=float32]
Coordinates:
  * time       (time) int64 0 1 2 3 4 5 6 ... 1994 1995 1996 1997 1998 1999 2000
  * latitude   (latitude) float32 -90.0 -88.0 -86.0 -84.0 ... 86.0 88.0 90.0
  * longitude  (longitude) float32 0.0 2.0 4.0 6.0 ... 352.0 354.0 356.0 358.0
Dimensions without coordinates: MCrun
Attributes:
    var_desc:       Air temperature
    long_name:      Air Temperature at Surface
    standard_name:  air_temperature
    units:          degK
    level_desc:     2m
    statistic:      Ensemble Mean
    GRIB_id:        11
    GRIB_name:      TMP
    valid_range:    [-20.  20.]
    dataset:        NOAA Last Millennium Reanalysis version 1.1 Annual Averages
    actual_range:   [-10.047585    7.4486136]

In [ ]:
xrft.fft(T, dimstr='time', detrend='constant',)